In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
def data():
    global df
    df=pd.read_csv("intrusion_dataset.csv")
    return df

In [3]:
data()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239,0,udp,domain_u,SF,45,45,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
1240,0,tcp,ftp_data,SF,74,0,0,0,0,0,...,109,0.17,0.02,0.17,0.02,0.07,0.00,0.01,0.00,normal
1241,0,udp,domain_u,SF,43,68,0,0,0,0,...,200,0.78,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal
1242,2,tcp,smtp,SF,1243,366,0,0,0,0,...,115,0.71,0.02,0.01,0.00,0.00,0.00,0.00,0.00,normal


In [4]:
def stats():
    global df
    stats = df.describe()
    return stats

In [5]:
stats()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,1244.000000,1.244000e+03,1244.000000,1244.0,1244.000000,1244.0,1244.000000,1244.000000,1244.000000,1244.000000,...,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000
mean,373.723473,3.150584e+05,1469.859325,0.0,0.032154,0.0,0.155949,0.002412,0.381029,0.029743,...,181.821543,109.704180,0.499606,0.082018,0.156672,0.034936,0.296125,0.293344,0.117283,0.127926
std,3023.607055,1.082347e+07,6295.881320,0.0,0.306426,0.0,1.880104,0.063377,0.485835,0.572850,...,99.443214,110.139753,0.450397,0.180990,0.317404,0.117114,0.451834,0.452973,0.301281,0.328398
min,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,80.500000,9.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3.200000e+01,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,255.000000,51.500000,0.370000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.715000e+02,455.750000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,...,255.000000,255.000000,1.000000,0.070000,0.080000,0.020000,1.000000,1.000000,0.000000,0.000000
max,41802.000000,3.817091e+08,148722.000000,0.0,3.000000,0.0,30.000000,2.000000,1.000000,19.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
def preprocess():
    global df
    df = df.drop(['protocol_type','service','flag'],axis=1)
    df=df.loc[:, (df==0).mean() < .7]
    df['label'] = df['label'].map({'anomaly':1,'normal':0})
    return df

In [7]:
preprocess()

,src_bytes,dst_bytes,logged_in,count,srv_count,serror_rate,srv_serror_rate,same_srv_rate,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,label
0,491,0,0,2,2,0.0,0.0,1.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0
1,146,0,0,13,1,0.0,0.0,0.08,0.15,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0
2,0,0,0,123,6,1.0,1.0,0.05,0.07,255,26,0.10,0.05,0.00,0.00,1.00,1.00,1
3,232,8153,1,5,5,0.2,0.2,1.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0
4,199,420,1,30,32,0.0,0.0,1.00,0.00,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239,45,45,0,107,112,0.0,0.0,1.00,0.00,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0
1240,74,0,1,2,4,0.0,0.0,1.00,0.00,121,109,0.17,0.02,0.17,0.02,0.07,0.00,0
1241,43,68,0,261,261,0.0,0.0,1.00,0.00,255,200,0.78,0.01,0.00,0.00,0.00,0.00,0
1242,1243,366,1,1,1,0.0,0.0,1.00,0.00,163,115,0.71,0.02,0.01,0.00,0.00,0.00,0


In [8]:
def Train_Test():
    global x_train, x_test, y_train, y_test
    x=df.iloc[:,:-1]
    y=df.iloc[:,-1]
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=25)
    return x_train,x_test,y_train,y_test

In [9]:
def RF():
    x_train,x_test,y_train,y_test=Train_Test()
    global new_x_train,new_x_test
    model= RandomForestClassifier(random_state=25)
    model= model.fit(x_train,y_train)
    features = pd.DataFrame()
    features['Feature'] = x_train.columns
    features['Importance'] = model.feature_importances_
    features.sort_values(by=['Importance'], ascending=False, inplace=True)
    selector = SelectFromModel(model, prefit=True)
    train_reduced = selector.transform(x_train)
    columns=x_train.columns[selector.get_support()]
    new_x_train=pd.DataFrame(train_reduced,columns=columns)
    test_reduced = selector.transform(x_test)
    new_x_test=pd.DataFrame(test_reduced,columns=columns)   
    rf = RandomForestClassifier(random_state=25)
    rf.fit(new_x_train, y_train)
    pred=rf.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = rf.predict(new_x_test).astype(int)
    df_output = pd.DataFrame()    
    df_output['Network_Intrusion_Predicted'] = np.vectorize(lambda s: 'Anomaly' if s==1 else 'Normal')(output)
    final_pred=pd.concat([new_x_test,df_output],axis=1 )
    final_pred.to_csv('Network_Prediction_submission@RF.csv',index=False)
    print("selected Top features:\n\n",features[:5])
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [10]:
RF()

selected Top features:

                    Feature  Importance
0                src_bytes    0.242506
1                dst_bytes    0.190175
10      dst_host_srv_count    0.104591
7            same_srv_rate    0.082484
11  dst_host_same_srv_rate    0.080019
Confusion Matrix:

 [[125   4]
 [  1 119]]

Accuracy Score:

 97.992 %

Predicted Values on the Test data:

 [1 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 1 1
 1 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1
 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1 1 1
 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1
 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0
 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 1
 1 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1]

Classification Report:

               precision    recall  f1-score   support

           0       0.99      0.97      0.98  

In [11]:
def KNN():
    x_train,x_test,y_train,y_test=Train_Test()
    global new_x_train,new_x_test
    model = KNeighborsClassifier()
    model.fit(new_x_train, y_train)
    pred=model.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = model.predict(new_x_test).astype(int)
    df_output = pd.DataFrame()    
    df_output['Network_Intrusion_Predicted'] = np.vectorize(lambda s: 'Anomaly' if s==1 else 'Normal')(output)
    final_pred=pd.concat([new_x_test,df_output],axis=1 )
    final_pred.to_csv('Network_Prediction_submission@KNN.csv',index=False)    
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [12]:
KNN()

Confusion Matrix:

 [[120   9]
 [  4 116]]

Accuracy Score:

 94.7791 %

Predicted Values on the Test data:

 [1 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 1 1
 1 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1
 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0
 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1
 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0
 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 1
 1 1 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 0 1]

Classification Report:

               precision    recall  f1-score   support

           0       0.97      0.93      0.95       129
           1       0.93      0.97      0.95       120

    accuracy                           0.95       249
   macro avg       0.95      0.95      0.95       249
weighted avg       0.95      0.95      0.95       249


Predicted values on the New Da

In [13]:
def DT():
    x_train,x_test,y_train,y_test=Train_Test()
    global new_x_train,new_x_test
    model = DecisionTreeClassifier()
    model.fit(new_x_train, y_train)
    pred=model.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = model.predict(new_x_test).astype(int)
    df_output = pd.DataFrame()    
    df_output['Network_Intrusion_Predicted'] = np.vectorize(lambda s: 'Anomaly' if s==1 else 'Normal')(output)
    final_pred=pd.concat([new_x_test,df_output],axis=1 )
    final_pred.to_csv('Network_Prediction_submission@DT.csv',index=False)    
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [14]:
DT()

Confusion Matrix:

 [[128   1]
 [  3 117]]

Accuracy Score:

 98.3936 %

Predicted Values on the Test data:

 [1 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 1 1
 1 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1
 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0
 1 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 0 0
 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 1
 1 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1]

Classification Report:

               precision    recall  f1-score   support

           0       0.98      0.99      0.98       129
           1       0.99      0.97      0.98       120

    accuracy                           0.98       249
   macro avg       0.98      0.98      0.98       249
weighted avg       0.98      0.98      0.98       249


Predicted values on the New Da